In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv("IMDBDataset.csv")
data.head()
data.shape

(50000, 2)

In [3]:
type(data)


pandas.core.frame.DataFrame

In [4]:
data["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [5]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

In [6]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [7]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


2025-01-20 20:56:37.889868: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-20 20:56:37.898099: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-20 20:56:37.919675: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737386797.953434   15731 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737386797.963220   15731 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-20 20:56:38.001654: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [8]:
train_data, test_data = train_test_split(data, test_size = 0.2, random_state=42)

In [9]:
train_data.shape

(40000, 2)

In [10]:
test_data.shape

(10000, 2)

In [11]:
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(train_data["review"])

In [12]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)


In [13]:
X_train

array([[1935,    1, 1200, ...,  205,  351, 3856],
       [   3, 1651,  595, ...,   89,  103,    9],
       [   0,    0,    0, ...,    2,  710,   62],
       ...,
       [   0,    0,    0, ..., 1641,    2,  603],
       [   0,    0,    0, ...,  245,  103,  125],
       [   0,    0,    0, ...,   70,   73, 2062]], dtype=int32)

In [14]:
X_test

array([[   0,    0,    0, ...,  995,  719,  155],
       [  12,  162,   59, ...,  380,    7,    7],
       [   0,    0,    0, ...,   50, 1088,   96],
       ...,
       [   0,    0,    0, ...,  125,  200, 3241],
       [   0,    0,    0, ..., 1066,    1, 2305],
       [   0,    0,    0, ...,    1,  332,   27]], dtype=int32)

In [15]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [16]:
Y_train

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64

In [17]:
model = Sequential()
model.add(Embedding(input_dim =5000, output_dim = 128, input_length = 200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout = 0.2))
model.add(Dense(1, activation = "sigmoid"))

W0000 00:00:1737386814.103079   15731 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [18]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [19]:
model.compile(optimizer = "adam", loss="binary_crossentropy", metrics=["accuracy"])

In [20]:
model.fit(X_train, Y_train, epochs = 5, batch_size = 64, validation_split = 0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 108s 211ms/step - accuracy: 0.7235 - loss: 0.5430 - val_accuracy: 0.8049 - val_loss: 0.4263
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 84s 167ms/step - accuracy: 0.8389 - loss: 0.3772 - val_accuracy: 0.8245 - val_loss: 0.3987
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 79s 159ms/step - accuracy: 0.8443 - loss: 0.3630 - val_accuracy: 0.8597 - val_loss: 0.3342
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 79s 158ms/step - accuracy: 0.8769 - loss: 0.3111 - val_accuracy: 0.8584 - val_loss: 0.3300
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 79s 158ms/step - accuracy: 0.9000 - loss: 0.2546 - val_accuracy: 0.8755 - val_loss: 0.3027


In [21]:
model.save("model.h5")

In [22]:
import joblib
joblib.dump(tokenizer, "tokenizer.pkl")


['tokenizer.pkl']

In [23]:
loss, accuracy = model.evaluate(X_test, Y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 33ms/step - accuracy: 0.8800 - loss: 0.2940


In [24]:
print(loss)

0.29014867544174194


In [25]:
print(accuracy)

0.8820000290870667


In [26]:
def predictive_system(review):
  sequences = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequences, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [27]:
predictive_system("movie  is good")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step


'positive'

In [37]:
predictive_system("Product is bad nut i liked most")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step


'positive'

In [ ]:
predictive_system("it's a fake movie ")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step


'negative'

In [57]:
predictive_system("it's a good movie")    

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step


'positive'